## Preprocessing

In [69]:

%matplotlib
# Import our dependencies
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

Using matplotlib backend: Qt5Agg


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [70]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace = True)

In [71]:
# Determine the number of unique values in each column.
application_df.apply(lambda x: x.nunique())

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [72]:
# Look at APPLICATION_TYPE value counts for binning
application_df.APPLICATION_TYPE.value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [73]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Cutoff value is 528 and remaining is included in a group called 'Other'
application_types_to_replace = ['T2', 'T9', 'T12', 'T13', 'T14', 'T15', 'T17', 'T25', 'T29']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [74]:
# Look at CLASSIFICATION value counts for binning
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1580        1
C1900        1
C8210        1
C1370        1
C1732        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [75]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df.CLASSIFICATION.value_counts().loc[lambda x: x>1]


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1600        5
C1257        5
C0           3
C2710        3
C1260        3
C1234        2
C3200        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [76]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = ['C7000', 'C1700', 'C4000','C5000', 'C1270',
                              'C2700','C2800', 'C7100', 'C1300', 'C1280', 
                              'C1230', 'C1400', 'C7200', 'C2300', 'C1240',
                              'C8000', 'C7120', 'C1500', 'C6000', 'C1800', 
                              'C1250', 'C8200', 'C1278', 'C1238','C1235', 
                              'C1237', 'C7210', 'C4100', 'C1720', 'C2400',
                              'C1257', 'C1600', 'C2710','C0', 'C1260', 
                              'C1246', 'C1256', 'C1234', 'C3200', 'C1267',
                             'C4120', 'C2600', 'C2170', 'C1732', 'C1283', 
                              'C1580', 'C3700', 'C1245', 'C1570', 'C4500',
                              'C1236', 'C2561', 'C4200', 'C2570', 'C1900',
                              'C2190', 'C1248', 'C1728', 'C6100', 'C1370', 
                              'C8210', 'C2380', 'C5200', 'C2500', 'C1820',
                              'C2150']

# Replace in dataframe
for cln in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cln,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [77]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [90]:
# Split our preprocessed data into our features and target arrays
features = application_df.loc[:,application_df.columns != 'IS_SUCCESSFUL']
target = application_df.loc[:,application_df.columns == 'IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
# Use sklearn to split dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=128)

In [91]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [83]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=50, activation="relu", input_dim=43))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=14))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=14))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_57 (Dense)             (None, 50)                2200      
_________________________________________________________________
dense_58 (Dense)             (None, 10)                510       
_________________________________________________________________
dense_59 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_60 (Dense)             (None, 1)                 11        
Total params: 2,831
Trainable params: 2,831
Non-trainable params: 0
_________________________________________________________________


In [84]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [85]:
# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5765 - accuracy: 0.7159
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5551 - accuracy: 0.7283
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5520 - accuracy: 0.7290
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5499 - accuracy: 0.7300
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5494 - accuracy: 0.7303
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5484 - accuracy: 0.7317
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5472 - accuracy: 0.7315
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5468 - accuracy: 0.7306
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5465 - accuracy: 0.7320
Epoch 10/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5457 - accuracy: 0.7318

In [86]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5564 - accuracy: 0.7270
Loss: 0.5564177632331848, Accuracy: 0.7269970774650574


In [89]:
# Create a DataFrame containing training history
history_df = pd.DataFrame(fit_model.history, index=range(1,len(fit_model.history["loss"])+1))

# Plot the accuracy
history_df.plot(y="accuracy")
plt.show()

plt.savefig('model_accuracy.png')

In [88]:
# !pip install pyyaml h5py
# Export our model to HDF5 file
import os
import tensorflow.keras as keras
nn_model.save('nn_model.h5')
